# Комп'ютерний практикум

In [43]:
import hashlib as hl        # sha1
import secrets as rnd       # randbelow, choice
import os, sys              # output and logging
import numpy as np, math    # calculations

In [44]:
# Some hardcoded stuff specifically for sha1
text = ' \t\n\r\v\fabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
punct = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
ALPH = text + punct

PIA_CMP_PREF_C = 40 - 4    # 4 chars to compare in the end
BDA_CMP_PREF_C = 40 - 8    # 8 chars to compare in the end

PIA_CMP_PREF_B = 20 - 2    # 2 bytes to compare in the end
BDA_CMP_PREF_B = 20 - 4    # 4 bytes to compare in the end

N = 100                    # Iteration count

STD_OS = sys.stdout
# ------------------------------------------

BASE_MESSAGE_PARTS = ["Bondar", "Petro", "Olexandrovych", ""]

In [45]:
def gen_bm_sample(char_counts=[0, 0, 0, 0]):
    result_msg_parts = []

    for i in range(4):
        if char_counts[i] != 0:
            gen_part = str()
            for _ in range(char_counts[i]):
                gen_part += rnd.choice(ALPH[6:])
            result_msg_parts.append(gen_part)
        result_msg_parts.append(BASE_MESSAGE_PARTS[i])

    return "".join(result_msg_parts)

def random_modification(base: str):
    res = list(base)

    for i in range(len(res)):
        if rnd.randbelow(2) == 1:
            res[i] = rnd.choice(ALPH)

    return "".join(res)

def rand_count_vec():
    return ([rnd.randbelow(6),
             rnd.randbelow(6),
             rnd.randbelow(6),
             rnd.randbelow(6)])


In [46]:
def preimage_attack_1(base_message: str, log=False, log_output='preimage_attack_1.txt'):
    print(f"Base message is: {base_message}\n")
    # Formating parameters and output
    pw = len(base_message) + 6

    out = sys.stdout
    if log:
        out = open(log_output, 'w')
    #

    base_sha = hl.sha1(base_message.encode())
    
    out.write(f"{base_message:{pw}s} : {base_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{base_sha.hexdigest()[PIA_CMP_PREF_C:]}\n")
    if log: print(f"{base_message:{pw}s} : {base_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{base_sha.hexdigest()[PIA_CMP_PREF_C:]}")
  
    app = 0
    while True:
        app += 1

        msg = base_message + str(app)
        msg_sha = hl.sha1(msg.encode())

        out.write(f"{msg:{pw}s} : {msg_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{msg_sha.hexdigest()[PIA_CMP_PREF_C:]}")

        if msg_sha.digest()[PIA_CMP_PREF_B:] == base_sha.digest()[PIA_CMP_PREF_B:]:
            out.write(f"\tSecond preimage on {app}!")
            if log: 
                print(f"{msg:{pw}s} : {msg_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{msg_sha.hexdigest()[PIA_CMP_PREF_C:]}", end='')
                print(f"\tSecond preimage on {app}!")
            
            break
        else:
            out.write('\n')

    if out != sys.stdout:
        out.close()
    
    return app

In [47]:
def preimage_attack_2(base_message: str, log=False, log_output='preimage_attack_2.txt'):
    print(f"Base message is: {base_message}\n")
    # Formating parameters
    pw = len(base_message) + 8

    out = sys.stdout
    if log:
        out = open(log_output, 'w')
    #

    base_sha = hl.sha1(base_message.encode())

    out.write(f"{repr(base_message):{pw}s} : {base_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{base_sha.hexdigest()[PIA_CMP_PREF_C:]}\n")
    if log: print(f"{repr(base_message):{pw}s} : {base_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{base_sha.hexdigest()[PIA_CMP_PREF_C:]}")

    itr = 0
    while True:
        itr += 1
        msg = random_modification(base_message)
        msg_sha = hl.sha1(msg.encode())

        out.write(f"{repr(msg):{pw}s} : {msg_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{msg_sha.hexdigest()[PIA_CMP_PREF_C:]}")

        if msg_sha.digest()[PIA_CMP_PREF_B:] == base_sha.digest()[PIA_CMP_PREF_B:] and msg != base_message:
            out.write(f"\tSecond preimage in {itr} iterations!\n")
            if log: 
                print(f"{repr(msg):{pw}s} : {msg_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{msg_sha.hexdigest()[PIA_CMP_PREF_C:]}", end='')
                print(f"\tSecond preimage in {itr} iterations!")

            break
        else:
            out.write('\n')

    if out != sys.stdout:
        out.close()
    
    return itr
        

### Атака пошуку другого прообразу 1

In [48]:
def birthday_attack_1(base_message: str, log=False, log_output='birthday_attack_1.txt'):
    print(f"Base message is: {base_message}\n")
    # Formating parameters
    pw = len(base_message) + 6

    out = sys.stdout
    if log:
        out = open(log_output, 'w')
    #

    base_sha = hl.sha1(base_message.encode())

    out.write(f"{base_message:{pw}s} : {base_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{base_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
    if log: print(f"{base_message:{pw}s} : {base_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{base_sha.hexdigest()[BDA_CMP_PREF_C:]}")

    prev_dict = {base_sha.digest()[BDA_CMP_PREF_B:]: base_message}
    app = 0
    while True:
        app += 1

        msg = base_message + str(app)
        msg_sha = hl.sha1(msg.encode())

        if msg_sha.digest()[BDA_CMP_PREF_B:] in prev_dict.keys():
            col = prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]]
            col_sha = hl.sha1(col.encode())
            
            out.write(f"\nCollision found in {app} iterations!\n")
            out.write(f"Msg: {msg:{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
            out.write(f"Col: {col:{pw}s} : {col_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{col_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
            if log: 
                print(f"\nCollision found in {app} iterations!")
                print(f"Msg: {msg:{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}")
                print(f"Col: {col:{pw}s} : {col_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{col_sha.hexdigest()[BDA_CMP_PREF_C:]}")

            break
        else:
            out.write(f"{msg:{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
            prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]] = msg
    
    if out != sys.stdout:
        out.close()
    
    return app

In [49]:
def birthday_attack_2(base_message: str, log=False, log_output='birthday_attack_2.txt'):
    print(f"Base message is: {base_message}\n")
    # Formating parameters
    pw = len(base_message) + 8

    out = sys.stdout
    if log:
        out = open(log_output, 'w')
    #

    base_sha = hl.sha1(base_message.encode())

    out.write(f"{repr(base_message):{pw}s} : {base_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{base_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
    if log: print(f"{repr(base_message):{pw}s} : {base_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{base_sha.hexdigest()[BDA_CMP_PREF_C:]}")

    prev_dict = {base_sha.digest()[BDA_CMP_PREF_B:]: base_message}
    itr = 0
    while True:
        itr += 1
        msg = random_modification(base_message)
        msg_sha = hl.sha1(msg.encode())

        if msg_sha.digest()[BDA_CMP_PREF_B:] in prev_dict.keys() and msg != base_message:
            col = prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]]
            col_sha = hl.sha1(col.encode())

            out.write(f"\nCollision found in {itr} iterations!\n")
            out.write(f"Msg: {repr(msg):{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
            out.write(f"Col: {repr(col):{pw}s} : {col_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{col_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
            if log: 
                print(f"\nCollision found in {itr} iterations!")
                print(f"Msg: {repr(msg):{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}")
                print(f"Col: {repr(col):{pw}s} : {col_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{col_sha.hexdigest()[BDA_CMP_PREF_C:]}")
                
            break
        else:
            out.write(f"{repr(msg):{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")
            prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]] = msg

    if out != sys.stdout:
        out.close()
    
    return itr

## Запуск атак

### Атака пошуку другого прообразу 1

#### Перша атака

In [50]:
preimage_attack_1(gen_bm_sample([5, 0, 0, 0]), True, os.devnull)

Base message is: 0[6s,BondarPetroOlexandrovych

0[6s,BondarPetroOlexandrovych       : 1b8817784ae372ce25d044090791af69343f	2d8a


0[6s,BondarPetroOlexandrovych89926  : 4d203d9db2ed0df2d0d34ebd711b7fe8ee61	2d8a	Second preimage on 89926!


89926

In [51]:
stats = []

with open(os.devnull, 'w') as f:
    sys.stdout = f
    for i in range(N):
        print(i)
        stats.append(preimage_attack_1(gen_bm_sample(rand_count_vec()), True, os.devnull))
    sys.stdout = STD_OS

sum_itr = sum(stats)
mean_val = np.mean(stats)
variance = np.var(stats)

print(f"Sum of iterations: {sum_itr}")
print(f"Mean: {mean_val}")
print(f"Variance: {variance}")

Sum of iterations: 6155243
Mean: 61552.43
Variance: 3399364161.2051


### Атака пошуку другого прообразу 2

In [52]:
preimage_attack_2(gen_bm_sample([0, 5, 0, 0]), True, os.devnull)

Base message is: BondarZ1n#`PetroOlexandrovych

'BondarZ1n#`PetroOlexandrovych'       : 6ed2bf1da5f0ddc51d1838653d521aac0314	7a6c
'B(vdarv1%#`P>tro9+eJardro\x0c_cV'    : 89398e888049e9809de32e6a8c34baccd9a8	7a6c	Second preimage in 58639 iterations!


58639

In [53]:
stats = []

with open(os.devnull, 'w') as f:
    sys.stdout = f
    for i in range(N):
        print(i)
        stats.append(preimage_attack_2(gen_bm_sample(rand_count_vec()), True, os.devnull))
    sys.stdout = STD_OS

sum_itr = sum(stats)
mean_val = np.mean(stats)
variance = np.var(stats)

print(f"Sum of iterations: {sum_itr}")
print(f"Mean: {mean_val}")
print(f"Variance: {variance}")

Sum of iterations: 6997988
Mean: 69979.88
Variance: 4610004413.3656


### Атака днів народжень 1

In [54]:
birthday_attack_1(gen_bm_sample([0, 0, 5, 0]), True, os.devnull)

Base message is: BondarPetroi(DIFOlexandrovych

BondarPetroi(DIFOlexandrovych       : 82955d56ee39dabbb2cd75d7a188965b	0f105368

Collision found in 93216 iterations!
Msg: BondarPetroi(DIFOlexandrovych93216  : 00213dc7807b501526de3d8f74ed4a5c	381dc2cd
Col: BondarPetroi(DIFOlexandrovych55724  : ab04e2528061e2d2b8721119d0e519c5	381dc2cd


93216

In [55]:
stats = []

with open(os.devnull, 'w') as f:
    sys.stdout = f
    for i in range(N):
        print(i)
        stats.append(birthday_attack_1(gen_bm_sample(rand_count_vec()), True, os.devnull))
    sys.stdout = STD_OS

sum_itr = sum(stats)
mean_val = np.mean(stats)
variance = np.var(stats)

print(f"Sum of iterations: {sum_itr}")
print(f"Mean: {mean_val}")
print(f"Variance: {variance}")

Sum of iterations: 8212658
Mean: 82126.58
Variance: 2058342953.6436002


### Атака днів народжень 2

In [56]:
birthday_attack_2(gen_bm_sample([0, 0, 0, 5]), True, os.devnull)

Base message is: BondarPetroOlexandrovychw!lry

'BondarPetroOlexandrovychw!lry'       : f84e71ea31b9a4058c0a5d67f4cad5ac	1ce8a7ae

Collision found in 34961 iterations!
Msg: 'B&(darPeKr$<lex\nndcUvU\x0bB[!$rL'   : 635a7a56d57f06bcb5a607d96bad7511	a5f63ec2
Col: 'tvab_rP3tr~O?REv%d\n\tvy@h$cA/P'     : d7e7078ac8082fd4d862797032689533	a5f63ec2


34961

In [57]:
stats = []

with open(os.devnull, 'w') as f:
    sys.stdout = f
    for i in range(N):
        print(i)
        stats.append(birthday_attack_2(gen_bm_sample(rand_count_vec()), True, os.devnull))
    sys.stdout = STD_OS

sum_itr = sum(stats)
mean_val = np.mean(stats)
variance = np.var(stats)

print(f"Sum of iterations: {sum_itr}")
print(f"Mean: {mean_val}")
print(f"Variance: {variance}")

Sum of iterations: 7987870
Mean: 79878.7
Variance: 2100377687.2499993
